**Current Variables**
- country
- year 
- suicide number 
- age category
- gdp
- gdp per capita

**Independent variables**
- social support
- international migrants

**to merge**
- share of population with mental health disorders
- literacy
- rural population
- unemployment
- population over 65 yo
- fertility rate
- health expenditure per capita
- depression rates

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
master = pd.read_csv('../archive/master.csv')
depression = pd.read_csv('../data/depression.csv', skiprows = 1)
elderly = pd.read_csv('../data/elderly.csv', skiprows = 5)
fertility = pd.read_csv('../data/fertility.csv', skiprows = 5)
health_exp = pd.read_csv('../data/health_exp.csv', skiprows = 5)
literacy = pd.read_csv('../data/literacy.csv', skiprows = 5)
m_health = pd.read_csv('../data/m_health.csv', skiprows = 1)
rural = pd.read_csv('../data/rural.csv', skiprows = 5)
unemployment = pd.read_csv('../data/unemployment.csv', skiprows = 5)

In [3]:
#merge our world in data dataset
owid = pd.merge(m_health, depression, on = ['Year', 'Entity'], how = 'outer' )
owid = owid[['Entity', 'Year', 
             'Prevalence - Mental disorders - Sex: Both - Age: Age-standardized (Percent)', 
            'Prevalence - Depressive disorders - Sex: Both - Age: Age-standardized (Percent)']]
owid.columns = ['country', 'year', 'pct_mdisorder', 'pct_depression']

In [4]:
#transform world bank data to fit master and owid
wb = [elderly, fertility, health_exp, literacy, rural, unemployment]
new_wb = []
for i in wb:
    #only keep country, indicator and dates as columns
    i = i[['Country Name', 'Indicator Name'] + [str(i) for i in list(range(1960, 2021))]]
    i.columns = ['country', 'indicator'] + [str(i) for i in list(range(1960, 2021))]
    new = pd.melt(i, id_vars = ['country'], value_vars = [str(i) for i in list(range(1960, 2021))], ignore_index = False)
    new = new.sort_values('country')
    new.columns = ['country', 'year', str(i['indicator'].loc[0])]
    new_wb.append(new)


In [5]:
#merge everything
import functools
wb_owid = new_wb + [owid] + [master]
df_merged = functools.reduce(lambda  left,right: pd.merge(left,right,on=['country', 'year'],
                                            how='outer'), wb_owid).fillna('void')
df_merged.to_csv('../archive/merged_master.csv')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat